In [1]:
import pandas as pd

# 한글 폰트 깨짐 방지
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

#에러 무시
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib

from sklearn.tree import export_graphviz
import graphviz

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import scale, minmax_scale, robust_scale

import xgboost as xgb

import seaborn as sns

plt.rcParams['font.family'] = 'NanumGothic'


In [2]:
df = pd.read_csv('82년 ~ 18년 타자 성적.csv', encoding = 'CP949')

In [3]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [4]:
df.corr()['OPS'].sort_values()

시즌     -0.089117
나이      0.178995
희타      0.283392
도루      0.318953
고4      0.343591
3타      0.353408
도실      0.380572
사구      0.452578
희비      0.466994
홈런      0.503492
병살      0.515106
WAR+    0.530023
삼진      0.568542
볼넷      0.575207
2타      0.584528
타점      0.587256
득점      0.587326
타수      0.605591
안타      0.607215
타석      0.607968
루타      0.609306
G       0.629175
wRC+    0.840125
출루      0.901480
wOBA    0.944047
타율      0.967631
장타      0.982626
OPS     1.000000
Name: OPS, dtype: float64

In [5]:
pd.read_csv('타자 기본 - test.csv', encoding = 'CP949')

,연도,선수코드,경기,타석,타수,타율,안타,홈런,루타,장타율,희생플라이,볼넷,삼진,고의사구,사구,병살타,출루율,OPS
0,2018,60100,70,169,152,0.243,37,3,63,0.414,1,12,36,0,4,5,0.314,NaN
1,2018,60184,6,10,10,0.200,2,0,3,0.300,0,0,3,0,0,0,0.200,NaN
2,2018,60288,1,1,1,0.000,0,0,0,0.000,0,0,0,0,0,0,0.000,NaN
3,2018,60343,83,174,162,0.216,35,8,63,0.389,0,8,49,0,3,3,0.266,NaN
4,2018,60456,4,8,6,0.500,3,0,3,0.500,0,2,0,0,0,0,0.625,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1093,2021,79290,4,8,8,0.125,1,0,1,0.125,0,0,2,0,0,0,0.125,NaN
1094,2021,79365,73,261,219,0.265,58,16,117,0.534,1,32,69,0,6,5,0.372,NaN
1095,2021,79402,70,278,237,0.211,50,1,61,0.257,0,30,26,0,3,7,0.307,NaN
1096,2021,79456,62,135,124,0.274,34,0,38,0.306,2,7,11,1,1,5,0.313,NaN


In [6]:
df = pd.read_csv('82년 ~ 18년 타자 성적 - train.csv', encoding = 'CP949')
df_t = pd.read_csv('타자 기본 - test.csv', encoding = 'CP949')

In [7]:
df_raw_y = df[['OPS']]# 목표변수
df_raw_x = df[['경기','타석', '타수', '타율', '안타', '홈런', '루타', '볼넷', '삼진', '병살타']] # 독립변수

In [9]:
# train, test set 나누기
df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df_raw_x, df_raw_y, test_size = 0.2, random_state = 1234)

print('train data x size : {}'.format(df_train_x.shape))
print('train data y size : {}'.format(df_train_y.shape))
print('test data x size : {}'.format(df_test_x.shape))
print('test data y size : {}'.format(df_test_y.shape))

train data x size : (6846, 10)
train data y size : (6846, 1)
test data x size : (1712, 10)
test data y size : (1712, 1)


In [10]:
# 기본 성능 모델
gb_uncustomized = xgb.XGBRegressor(random_state = 1234)
gb_uncustomized.fit(df_train_x, df_train_y)

print('Score on trainning set : {:3f}'.format(gb_uncustomized.score(df_train_x, df_train_y)))
print('Score on test set : {:3f}'.format(gb_uncustomized.score(df_test_x, df_test_y)))

Score on trainning set : 0.998698
Score on test set : 0.989237


In [11]:
# 최종 모델
# gblinear : 선형모형
# gbtree : 의사결정기반모형
# dart : 드롭아웃 regression trees

xgb_final = xgb.XGBRegressor(random_state = 1234, n_estimators = 400, subsample = 0.7,
                                  colsample_bytree = 1, colsample_bylevel = 0.7, max_depth = 3)

xgb_final.fit(df_train_x, df_train_y)

print('Score on training set: {:.3f}'.format(xgb_final.score(df_train_x, df_train_y)))
print('Score on test set: {:.3f}'.format(xgb_final.score(df_test_x, df_test_y)))

Score on training set: 0.997
Score on test set: 0.991


In [12]:
df_t = df_t[['경기','타석', '타수', '타율', '안타', '홈런', '루타', '볼넷', '삼진', '병살타']]

In [13]:
pred = xgb_final.predict(df_t)

In [14]:
pred

array([0.6925855 , 0.5003788 , 0.00161074, ..., 0.56422246, 0.6408865 ,
       0.8797321 ], dtype=float32)

In [15]:
df_tt = pd.read_csv('타자 기본 - test.csv', encoding = 'CP949')

In [16]:
df_tt['OPS'] = pred.round(3)

In [17]:
df_tt.to_csv('타자 기본 - result.csv', encoding = 'CP949')

In [18]:
df_basic = pd.read_csv('타자 기본.csv', encoding = 'CP949')

In [19]:
# 평균 절대 오차
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(df_tt['OPS'], df_basic['OPS'])
mae

0.0201475404359329

In [20]:
# 평균 제곱근 오차
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(df_tt['OPS'], df_basic['OPS'])
rmse

0.002584316838124197

https://wotres.tistory.com/entry/%ED%9A%8C%EA%B7%80-%EC%97%B0%EC%86%8D%ED%98%95-%EB%B3%80%EC%88%98-%EC%B8%A1%EC%A0%95%ED%95%98%EB%8A%94%EB%B2%95-MSE-RMSE-MAE-MPE-MAPE-in-python